In [1]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Paths to directories
images_dir = Path("images")
keypoints_dir = Path("NewOutputs/Keypoints")
matches_dir = Path("NewOutputs/Matches")

# Output video path
output_video_path = "keypoints_flow_video.mp4"

# Load image paths
image_paths = sorted(images_dir.glob("*.png"))
keypoints_path = sorted(keypoints_dir.glob("*.txt"))
matches_path = sorted(matches_dir.glob("*.txt"))

# Function to load keypoints from file
def load_keypoints(filepath):
    return np.loadtxt(filepath, dtype=float)

# Function to load matches from file
def load_matches(filepath):
    return np.loadtxt(filepath, dtype=int)

# Function to assign unique colors
def generate_colors(num_colors):
    np.random.seed(42)
    return np.random.randint(0, 255, size=(num_colors, 3))

# Initialize tracking data structures
tracked_keypoints = {}

# Initialize colors for keypoints
num_colors = 2048  # Assuming a maximum of 2048 keypoints
colors = generate_colors(num_colors)

# Create a VideoWriter object
frame = cv2.imread(str(image_paths[0]))
height, width, _ = frame.shape
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 1, (width * 2, height))

if not video_writer.isOpened():
    print("Error: VideoWriter not opened")
    exit()

# Iterate through image pairs
for i in tqdm(range(len(image_paths) - 1)):

    img1_path = image_paths[i]
    img2_path = image_paths[i + 1]

    # Load images
    img1 = cv2.imread(str(img1_path))
    img2 = cv2.imread(str(img2_path))

    # Check if images are loaded correctly
    if img1 is None:
        print(f"Failed to load image: {img1_path}")
        continue
    if img2 is None:
        print(f"Failed to load image: {img2_path}")
        continue

    # Load keypoints and matches
    kp1 = load_keypoints(keypoints_path[i])
    kp2 = load_keypoints(keypoints_path[i+1])
    matches = load_matches(matches_path[i])

    # Check if keypoints and matches are loaded correctly
    if kp1.size == 0 or kp2.size == 0 or matches.size == 0:
        print(f"Failed to load keypoints or matches for images {i} and {i+1}")
        continue

    # Update tracking information
    new_tracked_keypoints = {}
    for idx1, idx2 in matches:
        pt1 = (int(kp1[idx1][0]), int(kp1[idx1][1]))
        pt2 = (int(kp2[idx2][0]), int(kp2[idx2][1]))
        
        # Assign a unique color to the tracked keypoint
        if idx1 not in tracked_keypoints:
            color = colors[idx1 % num_colors]
            tracked_keypoints[idx1] = (pt1, color)
        else:
            color = tracked_keypoints[idx1][1]

        # Update new tracked keypoints
        new_tracked_keypoints[idx2] = (pt2, color)

    # Draw keypoints and matches
    for pt, color in tracked_keypoints.values():
        cv2.circle(img1, pt, 3, color.tolist(), -1)

    for pt, color in new_tracked_keypoints.values():
        cv2.circle(img2, pt, 3, color.tolist(), -1)

    for (idx1, idx2) in matches:
        pt1 = (int(kp1[idx1][0]), int(kp1[idx1][1]))
        pt2 = (int(kp2[idx2][0]), int(kp2[idx2][1]))
        color = tracked_keypoints[idx1][1]
        cv2.line(img1, pt1, pt2, color.tolist(), 1)
        cv2.line(img2, pt1, pt2, color.tolist(), 1)

    # Combine images side by side
    combined_image = np.hstack((img1, img2))

    # Debug: Save the frame as an image for verification
    cv2.imwrite(f"frame_{i}.png", combined_image)

    # Write frame to video
    video_writer.write(combined_image)

    # Debug: Check if the frame is being written
    print(f"Frame {i} written to video")

    # Update tracked keypoints
    tracked_keypoints = new_tracked_keypoints

# Release the VideoWriter object
video_writer.release()
print(f"Video saved to {output_video_path}")


  0%|▏                                          | 2/419 [00:00<01:03,  6.59it/s]

Frame 0 written to video
Frame 1 written to video


  1%|▍                                          | 4/419 [00:00<01:05,  6.37it/s]

Frame 2 written to video
Frame 3 written to video


  1%|▌                                          | 6/419 [00:00<01:04,  6.43it/s]

...

Frame 414 written to video
Frame 415 written to video


100%|████████████████████████████████████████▉| 418/419 [01:02<00:00,  6.74it/s]

Frame 416 written to video
Frame 417 written to video


100%|█████████████████████████████████████████| 419/419 [01:02<00:00,  6.71it/s]

Frame 418 written to video
Video saved to keypoints_flow_video.mp4


In [2]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Paths to directories
images_dir = Path("images")
keypoints_dir = Path("NewOutputs/Keypoints")
matches_dir = Path("NewOutputs/Matches")

# Output video path
output_video_path = "keypoints_flow_video.mp4"

# Load image paths
image_paths = sorted(images_dir.glob("*.png"))
keypoints_path = sorted(keypoints_dir.glob("*.txt"))
matches_path = sorted(matches_dir.glob("*.txt"))

# Function to load keypoints from file
def load_keypoints(filepath):
    return np.loadtxt(filepath, dtype=float)

# Function to load matches from file
def load_matches(filepath):
    return np.loadtxt(filepath, dtype=int)

# Function to assign unique colors
def generate_colors(num_colors):
    np.random.seed(42)
    return np.random.randint(0, 255, size=(num_colors, 3))

# Initialize tracking data structures
tracked_keypoints = {}

# Initialize colors for keypoints
num_colors = 2048  # Assuming a maximum of 2048 keypoints
colors = generate_colors(num_colors)

# Create a VideoWriter object
frame = cv2.imread(str(image_paths[0]))
height, width, _ = frame.shape
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))

if not video_writer.isOpened():
    print("Error: VideoWriter not opened")
    exit()

# Iterate through image pairs
for i in tqdm(range(len(image_paths))):

    img_path = image_paths[i]

    # Load images
    img = cv2.imread(str(img_path))

    # Check if images are loaded correctly
    if img is None:
        print(f"Failed to load image: {img_path}")
        continue

    # Load keypoints and matches
    kp = load_keypoints(keypoints_path[i])

    # Check if keypoints are loaded correctly
    if kp.size == 0:
        print(f"Failed to load keypoints for image {i}")
        continue

    # Draw keypoints
    for idx, (x, y) in enumerate(kp):
        cv2.circle(img, (int(x), int(y)), 3, colors[idx % num_colors].tolist(), -1)

    # Draw trajectories
    if i > 0:
        prev_kp = load_keypoints(keypoints_path[i - 1])
        matches = load_matches(matches_path[i - 1])
        
        for (idx1, idx2) in matches:
            pt1 = (int(prev_kp[idx1][0]), int(prev_kp[idx1][1]))
            pt2 = (int(kp[idx2][0]), int(kp[idx2][1]))
            color = colors[idx1 % num_colors].tolist()
            cv2.line(img, pt1, pt2, color, 2)

    # Write frame to video
    video_writer.write(img)

    # Debug: Save the frame as an image for verification
    cv2.imwrite(f"frame_{i}.png", img)

# Release the VideoWriter object
video_writer.release()
print(f"Video saved to {output_video_path}")


100%|█████████████████████████████████████████| 420/420 [00:32<00:00, 12.79it/s]

Video saved to keypoints_flow_video.mp4


In [7]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Paths to directories
images_dir = Path("images")
keypoints_dir = Path("NewOutputs/Keypoints")
matches_dir = Path("NewOutputs/Matches")

# Output video path
output_video_path = "keypoints_full_trajectory_video.mp4"

# Load image paths
image_paths = sorted(images_dir.glob("*.png"))
keypoints_path = sorted(keypoints_dir.glob("*.txt"))
matches_path = sorted(matches_dir.glob("*.txt"))

# Function to load keypoints from file
def load_keypoints(filepath):
    return np.loadtxt(filepath, dtype=float)

# Function to load matches from file
def load_matches(filepath):
    return np.loadtxt(filepath, dtype=int)

# Function to assign unique colors
def generate_colors(num_colors):
    np.random.seed(42)
    return np.random.randint(0, 255, size=(num_colors, 3))

# Initialize colors for keypoints
num_colors = 2048  # Assuming a maximum of 2048 keypoints
colors = generate_colors(num_colors)

# Create a VideoWriter object
frame = cv2.imread(str(image_paths[0]))
height, width, _ = frame.shape
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 5, (width, height))

if not video_writer.isOpened():
    print("Error: VideoWriter not opened")
    exit()

# Initialize dictionary to keep track of keypoints trajectories
trajectories = {}

# Iterate through image pairs
for i in tqdm(range(len(image_paths))):
    img_path = image_paths[i]

    # Load images
    img = cv2.imread(str(img_path))

    # Check if images are loaded correctly
    if img is None:
        print(f"Failed to load image: {img_path}")
        continue

    # Load keypoints
    kp = load_keypoints(keypoints_path[i])

    # Check if keypoints are loaded correctly
    if kp.size == 0:
        print(f"Failed to load keypoints for image {i}")
        continue

    # Initialize trajectories for the first frame
    if i == 0:
        for idx, (x, y) in enumerate(kp):
            trajectories[idx] = [(int(x), int(y))]

    # Update trajectories for subsequent frames
    if i > 0:
        prev_kp = load_keypoints(keypoints_path[i - 1])
        matches = load_matches(matches_path[i - 1])
        
        new_trajectories = {}
        for (idx1, idx2) in matches:
            pt1 = (int(prev_kp[idx1][0]), int(prev_kp[idx1][1]))
            pt2 = (int(kp[idx2][0]), int(kp[idx2][1]))
            if idx1 in trajectories:
                new_trajectories[idx2] = trajectories[idx1] + [pt2]
            else:
                new_trajectories[idx2] = [pt2]

        trajectories = new_trajectories

    # Draw keypoints
    for idx, (x, y) in enumerate(kp):
        cv2.circle(img, (int(x), int(y)), 3, colors[idx % num_colors].tolist(), -1)

    # Draw trajectories
    for idx, points in trajectories.items():
        color = colors[idx % num_colors].tolist()
        for j in range(len(points) - 1):
            cv2.line(img, points[j], points[j + 1], color, 1)

    # Write frame to video
    video_writer.write(img)

    # Debug: Save the frame as an image for verification
    cv2.imwrite(f"frame_{i}.png", img)

# Release the VideoWriter object
video_writer.release()
print(f"Video saved to {output_video_path}")


100%|█████████████████████████████████████████| 420/420 [00:51<00:00,  8.17it/s]

Video saved to keypoints_full_trajectory_video.mp4
